In [13]:
!pip install uvicorn nest_asyncio pyngrok fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: type

In [19]:
# importando bibliotecas utilizadas
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from tensorflow.keras.models import load_model

# carregando o dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# normalizando os valores dos pixels entre 0 e 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# definindo as categorias do CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [24]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10)
])

# Compilando o modelo
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Treinando o modelo
model.fit(train_images, train_labels, epochs=5,
          validation_data=(test_images, test_labels))

# Salvando o modelo treinado
model.save('cifar10_cnn_model.h5')

Epoch 1/5
1563/1563 [==============================] - 11s 5ms/step - loss: 1.6443 - accuracy: 0.3934 - val_loss: 1.2781 - val_accuracy: 0.5328
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2685 - accuracy: 0.5482 - val_loss: 1.0827 - val_accuracy: 0.6179
Epoch 3/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1117 - accuracy: 0.6116 - val_loss: 1.0198 - val_accuracy: 0.6358
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0027 - accuracy: 0.6486 - val_loss: 0.9471 - val_accuracy: 0.6650
Epoch 5/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9281 - accuracy: 0.6775 - val_loss: 0.8676 - val_accuracy: 0.7005


In [25]:
model = load_model('cifar10_cnn_model.h5')

# utilizando fast api para criar o endpoint
app = FastAPI()

# endpoint que recebe uma imagem como input e retorna o tipo dessa imagem
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        image_bytes = await file.read()

        img = tf.image.decode_image(image_bytes, channels=3)
        img = tf.image.resize(img, [32, 32])
        img = np.expand_dims(img, axis=0)

        predictions = model.predict(img)
        predicted_class = np.argmax(predictions[0])

        return JSONResponse({'Classe predita': class_names[predicted_class]})
    except Exception as e:
        return JSONResponse({'Error': str(e)}, status_code=500)

In [26]:
!ngrok authtoken 2h41yGKd4Low4y4GyBgOM9vG6LH_5A9uoraMacbiLovRbuXnV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# conectando e iniciando o servidor
ngrok_tunnel = ngrok.connect(8000)

print("Public URL:", ngrok_tunnel.public_url)

nest_asyncio.apply()

uvicorn.run(app, port=8000)

Public URL: https://0ea3-34-126-173-164.ngrok-free.app


INFO:     Started server process [242]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1/1 [==============================] - 0s 122ms/step
INFO:     177.92.77.170:0 - "POST /predict HTTP/1.1" 200 OK
